**Table of contents**<a id='toc0_'></a>    
- [移動ロボット](#toc1_)    
- [pybulletの起動](#toc2_)    
- [pybulletの初期設定](#toc3_)    
- [移動ロボットの生成](#toc4_)    
- [シミュレーションの実行](#toc5_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[移動ロボット](#toc0_)

本notebookでは2輪の移動ロボットを生成し、走行させる手順について解説します。

（pybulletで使用可能な関数がまとめられたマニュアルについては[こちら](https://github.com/bulletphysics/bullet3/blob/master/docs/pybullet_quickstartguide.pdf)を参照してください。）

# <a id='toc2_'></a>[pybulletの起動](#toc0_)

In [1]:
import pybullet
import pybullet_data
physicsClient = pybullet.connect(pybullet.GUI) 

pybullet build time: Nov 28 2023 23:45:17


startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Microsoft Corporation
GL_RENDERER=D3D12 (AMD Radeon Graphics)
GL_VERSION=4.2 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
GL_SHADING_LANGUAGE_VERSION=4.20
pthread_getconcurrency()=0
Version = 4.2 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
Vendor = Microsoft Corporation
Renderer = D3D12 (AMD Radeon Graphics)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


ven = Microsoft Corporation
ven = Microsoft Corporation


# <a id='toc3_'></a>[pybulletの初期設定](#toc0_)

In [5]:
pybullet.resetSimulation() # シミュレーション空間をリセット
pybullet.setAdditionalSearchPath(pybullet_data.getDataPath()) # pybulletに必要なデータへのパスを追加
pybullet.setGravity(0.0, 0.0, -9.8) # 地球上における重力に設定
timeStep = 1./240.
pybullet.setTimeStep(timeStep) # 1stepあたりに経過する時間の設定

#床の読み込み
planeId = pybullet.loadURDF("plane.urdf")

# GUIモードの際のカメラの位置などを設定
cameraDistance = 2.0
cameraYaw = 0.0 # deg
cameraPitch = -20 # deg
cameraTargetPosition = [0.0, 0.0, 0.0]
pybullet.resetDebugVisualizerCamera(cameraDistance, cameraYaw, cameraPitch, cameraTargetPosition)

# <a id='toc4_'></a>[移動ロボットの生成](#toc0_)
今回は、2輪ロボット`simple_two_wheel_car.urdf`を生成します。  
このロボットは下図のような構成になっています（センサーについては、`mobile_robot_sensor.ipynb`にて解説しています。）  

![](../images/mobile_robot1.png)

![](../images/mobile_robot2.png)

In [6]:
# ロボットの読み込み
carStartPos = [0, 0, 0.1]  # 初期位置(x,y,z)を設定
carStartOrientation = pybullet.getQuaternionFromEuler([0,0,0])  # 初期姿勢(roll, pitch, yaw)を設定
# urdfファイルのmeshはテクスチャが反映されないっぽいので各linkにrgbaタグで色を付けている
carId = pybullet.loadURDF("../urdf/simple_two_wheel_car.urdf",carStartPos, carStartOrientation)

# <a id='toc5_'></a>[シミュレーションの実行](#toc0_)
`setJointMotorControl2` では、指定したロボットの指定したジョイントを動作させることができます。  
ここでは、速度指令で動作させるため、第三引数に`VELOCITY_CONTROL`を指定しています。


In [7]:
import time
rightWheelJointIdx = 0
leftWheelJointIdx = 1
for i in range(500):
    # 速度10で前進
    pybullet.setJointMotorControl2(carId, leftWheelJointIdx, pybullet.VELOCITY_CONTROL, targetVelocity=10)
    pybullet.setJointMotorControl2(carId, rightWheelJointIdx, pybullet.VELOCITY_CONTROL, targetVelocity=10)

    # 速度5で後進
    # pybullet.setJointMotorControl2(carId, leftWheelJointIdx, pybullet.VELOCITY_CONTROL, targetVelocity=-5)
    # pybullet.setJointMotorControl2(carId, rightWheelJointIdx, pybullet.VELOCITY_CONTROL, targetVelocity=-5)

    # 右旋回
    # pybullet.setJointMotorControl2(carId, leftWheelJointIdx, pybullet.VELOCITY_CONTROL, targetVelocity=10)
    # pybullet.setJointMotorControl2(carId, rightWheelJointIdx, pybullet.VELOCITY_CONTROL, targetVelocity=2)

    # 左旋回
    # pybullet.setJointMotorControl2(carId, leftWheelJointIdx, pybullet.VELOCITY_CONTROL, targetVelocity=2)
    # pybullet.setJointMotorControl2(carId, rightWheelJointIdx, pybullet.VELOCITY_CONTROL, targetVelocity=10)

    pybullet.stepSimulation()
    time.sleep(timeStep)

: 